In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from matplotlib import style
from sklearn import preprocessing, cross_validation, svm
from sklearn import linear_model 

In [8]:
devel=pd.read_csv('ComParE2017_devel.csv')
train=pd.read_csv('ComParE2017_train.csv')

In [9]:
train.head() 

,1,2,3,4,5,6,7,8,9,10,...,6365,6366,6367,6368,6369,6370,6371,6372,6373,y
0,2.32,0.0769,0.5600,0.554,0.666,1.010,0.112,0.347,0.458,0.3630,...,0.879,1.98,2.02,-18.5,0.458,91.3,49.6,78.1,49.1,0
1,2.86,0.9780,0.8390,0.776,1.620,2.340,0.846,0.723,1.570,0.5590,...,0.490,2.45,2.60,-13.5,0.437,155.0,79.7,132.0,52.4,1
2,2.03,0.7420,0.0000,0.932,1.160,1.450,0.230,0.285,0.515,0.2640,...,0.646,2.77,2.59,13.0,0.477,133.0,66.7,90.0,44.6,1
3,2.26,0.2500,0.9750,0.467,0.661,1.100,0.194,0.439,0.634,0.2780,...,0.605,2.33,2.42,-15.3,0.337,99.4,51.0,85.7,53.4,1
4,2.41,0.5730,0.0142,0.231,0.483,0.806,0.252,0.323,0.575,0.0575,...,0.450,2.00,1.97,18.2,0.512,99.5,44.6,85.2,36.9,0


In [10]:
X_train = np.array(train.drop(['y'], 1))
X_val = np.array(devel.drop(['y'], 1))
X_train = preprocessing.scale(X_train)
X_val = preprocessing.scale(X_val)
#df.fillna(value=-99999, inplace=True)
#df.dropna(inplace=True)
y_train = np.array(train['y'])
y_val = np.array(devel['y'])

C:\Users\Rahul\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [11]:
x = np.array(X_train[:,6372])
x.dtype

dtype('float64')

In [13]:
X_t, X_v, y_t, y_v = cross_validation.train_test_split(X_train, y_train, test_size=0.4)


In [14]:
# Linear model
D = X_train.shape[1] #Number of features
K = max(y_train)+1 #Number of classes assuming class index starts from 0

# Start with an initialize parameters randomly
W = 0.01 * np.random.randn(D,K)
b = np.zeros((1,K))

# Initial values from hyperparameter
reg = 1e-3 # regularization strength

#For simplicity, we will not optimize this using grid search here.b#Perform batch SGD using backprop

#For simplicity we will take the batch size to be the same as number of examples
num_examples = X_train.shape[0]

#Initial value for the Gradient Descent Parameter
step_size = 1e-2 #Also called learning rate

#For simplicity, we will not hand tune this algorithm parameter as well.

# gradient descent loop
for i in range(10):
  
    # evaluate class scores, [N x K]
    scores = np.dot(X_train, W) + b 
  
    # compute the class probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
  
    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y_train])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W)
    loss = data_loss + reg_loss
    if i % 100 == 0:
        print("iteration:",i, " loss:",loss)
  
    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y_train] -= 1
    dscores /= num_examples
  
    # backpropate the gradient to the parameters (W,b)
    dW = np.dot(X_train.T, dscores)
    db = np.sum(dscores, axis=0, keepdims=True)
  
    dW += reg*W # regularization gradient
  
    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db


scores = np.dot(X_t, W) + b
predicted_class = np.argmax(scores, axis=1)
print("train accuracy:",(np.mean(predicted_class == y_t)))


scores = np.dot(X_v, W) + b
predicted_class = np.argmax(scores, axis=1)
print("val accuracy:",(np.mean(predicted_class == y_v)))


scores = np.dot(X_val, W) + b
predicted_class = np.argmax(scores, axis=1)
print("devel accuracy:",(np.mean(predicted_class == y_val)))

iteration: 0  loss: 0.777047802391
train accuracy: 0.686414253898
val accuracy: 0.688042752171
devel accuracy: 0.577183098592


In [23]:
scores = np.dot(X_val, W) + b
predicted_class = np.argmax(scores, axis=1)
print("devel accuracy:",(np.mean(predicted_class == y_val)))

devel accuracy: 0.0


In [20]:
scores = np.dot(X_val, W) + b
predicted_class_vl = np.argmax(scores, axis=1)
print("test accuracy:",(np.mean(predicted_class_vl == y_val)))

test accuracy: 0.600563380282


In [21]:
scores = np.dot(X_train, W) + b
predicted_class_tr = np.argmax(scores, axis=1)
print("train accuracy:",(np.mean(predicted_class_tr == y_train)))

train accuracy: 0.947087119188


In [22]:
from sklearn.metrics import recall_score
b=recall_score(y_train, predicted_class_tr, labels=None, pos_label=0, average='binary', sample_weight=None)
b

0.93680555555555556

In [25]:
b=recall_score(y_val, predicted_class_vl, labels=None, pos_label=0, average='binary', sample_weight=None)
b

0.51169590643274854

In [26]:
from sklearn.metrics import confusion_matrix
a=confusion_matrix(y_train, predicted_class_tr, labels=None)
a

array([[1349,   91],
       [ 107, 2195]])

In [27]:
from sklearn.metrics import confusion_matrix
a=confusion_matrix(y_val, predicted_class_vl, labels=None)
a

array([[ 700,  668],
       [ 750, 1432]])